In [1]:
from pyspark.sql import SparkSession

In [6]:
spark = SparkSession.builder.appName("JüngerAnalysis").getOrCreate()
spark.sparkContext.setLogLevel('WARN')

In [8]:
text = spark.read.text("waldgang.txt").rdd

In [9]:
text.take(10)

[Row(value='Эрнст Юнгер'),
 Row(value='Уход в лес'),
 Row(value='«здесь и сейчас»'),
 Row(value=''),
 Row(value='1'),
 Row(value=''),
 Row(value='Уход в Лес – отнюдь не идиллия скрывается за этим названием. Напротив, читатель должен быть готов к рискованной прогулке не только по проторённым тропам, но и, быть может, уводящей за пределы исследованного.'),
 Row(value=''),
 Row(value='Речь пойдёт о главном вопросе нашего времени, вопросе того рода, что всегда несёт в себе угрозу. Мы задаёмся многими вопросами, также как наши отцы и деды до нас. Между тем, безусловно, значительно изменился смысл того, что называют самим вопросом. Достаточно ли мы осознаём это?'),
 Row(value='')]

In [10]:
text = text.flatMap(lambda row: row.value.split())
text.take(20)

['Эрнст',
 'Юнгер',
 'Уход',
 'в',
 'лес',
 '«здесь',
 'и',
 'сейчас»',
 '1',
 'Уход',
 'в',
 'Лес',
 '–',
 'отнюдь',
 'не',
 'идиллия',
 'скрывается',
 'за',
 'этим',
 'названием.']

In [11]:
def normalize_string(text):
    symbols = []
    for s in text:
        if s.isalpha():
            symbols.append(s.lower())
    return ''.join(symbols)

text = text.map(normalize_string) \
           .filter(lambda word: word != '')
text.take(20)

['эрнст',
 'юнгер',
 'уход',
 'в',
 'лес',
 'здесь',
 'и',
 'сейчас',
 'уход',
 'в',
 'лес',
 'отнюдь',
 'не',
 'идиллия',
 'скрывается',
 'за',
 'этим',
 'названием',
 'напротив',
 'читатель']

In [28]:
text.groupBy(lambda word: word).mapValues(list).take(3)

[('эрнст', ['эрнст']), ('юнгер', ['юнгер']), ('уход', ['уход', 'уход'])]

In [32]:
text.groupBy(lambda word: word) \
    .mapValues(len) \
    .sortBy(lambda row: row[1], ascending=False) \
    .take(50)

[('в', 63),
 ('и', 52),
 ('что', 34),
 ('не', 30),
 ('на', 23),
 ('к', 21),
 ('мы', 18),
 ('это', 18),
 ('как', 16),
 ('можно', 14),
 ('но', 13),
 ('он', 13),
 ('с', 12),
 ('если', 12),
 ('от', 11),
 ('том', 10),
 ('всё', 10),
 ('при', 10),
 ('же', 10),
 ('только', 9),
 ('они', 9),
 ('то', 9),
 ('этом', 9),
 ('так', 9),
 ('по', 8),
 ('тем', 8),
 ('вопросы', 8),
 ('его', 8),
 ('бы', 8),
 ('быть', 7),
 ('может', 7),
 ('также', 7),
 ('из', 7),
 ('процентов', 7),
 ('или', 6),
 ('их', 6),
 ('этой', 6),
 ('будет', 6),
 ('за', 5),
 ('того', 5),
 ('скорее', 5),
 ('ответ', 5),
 ('избирательный', 5),
 ('бюллетень', 5),
 ('для', 5),
 ('даже', 5),
 ('выборы', 5),
 ('о', 4),
 ('всегда', 4),
 ('нас', 4)]